# Finding CRISPR arrays
## Specification
## Program Name: randomizedMotifSearch.py

## Input/Output: STDIN, STDOUT

## Options:

- -i iterations (int)
- -k motif length (int)
- -p pseudocount (float)

In [7]:
Ex: python randomizedMotifSearch.py -i 1000 -k 13 -p 1 < input.fa > output.out

Inspection Notebook example:
main ( "input.fa", options = [ "-i 1000", "-k 13", "-p 1"] )

SyntaxError: invalid syntax (3549359583.py, line 1)

## Background
CRISPR arrays exist in most archaeal species and many bacterial species. The arrays are encoded in chromosomal DNA as direct repeats (DR) that flank sequences (spacers) that provide immunity against invading plasmids, viral sequences, and other foreign DNA .. and RNA. These arrays can be short, comprising 1-2 spacers, or they can contain hundreds of spacer elements. The arrays are punctuated by these DR sequences. The promoter element seems to be upstream of the array, and initiates transcription of the associated RNA strand that is then cleaved and those spacers are loaded into protein machines that provide specific targeting of the associated invader should it be found. That targeting causes cleavage of the foreign DNA in known cases.

There are three main classes of CRISPR systems; these are type I,II and III, and are established based on the associated protein machines - actually the specific genes that encode for them. In the archaea, only type I and III have been found so far, while in bacteria, all three types have been seen. One of those types, type II, exists in a few species, including _Streptococcus pyogenes_, which encodes for a type II system. The specific gene that is used to classify this system is named Cas9.

We see cases where Cas genes, and sometimes CRISPR arrays, exist in a viral sequence. This, of course, opens many questions about the role of the CRISPR system with respect to viruses - why would a virus carry an immune defense that targets viral sequence? The existence of these arrays suggests that the CRISPR system itself might be mobile. How does the system get established, and how does the surrounding transcription machinery get encoded on the chromosome?

For this assignment, we will work to find the promoter motif. We would expect that this promoter motif and an associated B recognition element (BRE) would be present in archaeal species. What is that motif, and where is it located relative to transcription start?

We know little about the sequence specifics of the promoter element, though it seems to be about 13 nucleotides in length. In Pyrobaculum species, genes are packed quite tightly, so we can look upstream of the array by about 50 bases to see if a common motif sequence might exist. We need to assume that some mutations can be tolerated in that sequence.

## Null model
In prior years, we included an option that scrambled the input sequences before running to find a best score. This would effectively eliminate any signal that might be present in the data, while preserving the overall composition of the sequences. We would then compare the best score achieved without scrambling to the score achieved with scrambling. The information that remains in a scrambled set of sequences is only the base level composition of the sequences:

$Q=Pr_{s\in\lbrace{ACTG}\rbrace}(s)=\frac{count(s)}{N}$

We can then use relative entropy to find a score relative to the base level composition by:

$REscore = \sum_{i=0}^{cols}\sum^{j \in ACGT}P_{i}(j)\log_{2}(\frac{P_{i}(j)}{Q(j)} )$

## Assignment
__Write a BME205 style program (.py file)__ that is based on the Randomized Motif Search that presented in class/videos and is described in Chapter 2 of the text. Your program should accept a fasta file that contains multiple fasta records as input (STDIN), and your program will then output (STDOUT) the consensus sequence and associated profile score. The score will be the sum of encoding costs (entropies) across each position in the final profile. We will use pseudocounts for this assignment, as described in the text ( p. 86-89), and we will need an option to provide the number of pseudocounts (-p).

__Produce a notebook that should be fully runable for use during inspections.__

I find it easier to accumulate counts rather than calculating a probability based profile. This will also make it a bit easier to deal with pseudocounts when scoring your count-based profile.

$score = \sum_{i=0}^{cols}\sum^{j \in ACGT}P_{i}(j)\log_{2}(\frac{P_{i}(j)}{Q(j)} )$

Randomized Motif Search (p. 93) can be easily trapped in local minima, so we will need to iterate some number of times to find a trajectory that produces the best results. This will need to be an option (-i) that establishes the iteration number.

We also don't know the appropriate motif length that we are looking for, so we need an option (-k) to specify this.

The full command line string should then look something like this:

In [ ]:
randomizedMotifSearch.py -i=100000 -p=1 -k=13 <somefile.fa >someOutputFile.fa

your program will need to use standard OO style, include docstrings and be well commented. It must run! 

I will provide a few fasta files that present the upstream 50 bases from species of the Pyrobaculum group. It is possible that the promotor+BRE that we are looking for is conserved across these species, maybe not perfectly though. Providing more data to a single run may be useful.

I do have expression data for these species, so we know that not all of these arrays are functional - at least under the conditions when I grew them.

Your program should output the final score achieved, along with the associated consensus motif.

## Extra Credit
The following optional features would be useful and considered for a combined extra 5 points:

1) -g Use Gibbs sampling to find the optimal consensus motif.

2) -m Print the specific motif and the name of the contributing sequence for each of the participating fasta records.

Each of these options should only be true when the flag is specified by the user (ie. you shouldn't have to explicitly state True/False on the commandline).

Submit your working program to Canvas.

## Notes:

1) It is likely that we will need to amend this assignment as we work with it.

2) Sept-2021 eliminated the need for scrambling sequences by using relative entropy

## Inspection intro

- What data structure did you use to hold your sequence motifs (DNA)? 
        
        I used a list to hold the sequence motifs
- Is your profile  structure based on counts or frequencies? 
        
        My profile is based on the frequencies
- Where did you implement iterations (mostly why did you choose this)?
        
        I implemented the iterations in a separate function from the randomized search. The iterations function calls the randomized search function.
- If you implemented Gibbs Sampling, how did you implement recomputing of the profile and motif?

## Inspection Results
 - Added more detailed doc strings
 - Made relative entropy function calculation more explicit
 - Made randomized motif search function more explicit by creating intermediate variables.

In [10]:
import sys

class FastAreader:
    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''

        self.fname = fname
        self.fileH = None

    def doOpen(self):
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):

        header = ''
        sequence = ''

        with self.doOpen() as self.fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = self.fileH.readline()
            while not line.startswith('>'):
                line = self.fileH.readline()
            header = line[1:].rstrip()

            for line in self.fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence

class CommandLine():
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond. 
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.

    '''

    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''

        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
        )

        self.parser.add_argument('-i', '--iterations', nargs='?', default=1000, action='store', type=int,
                                help='min kMer size ')
        self.parser.add_argument('-k', '--motifSize', nargs='?', default=13, action='store', type=int,
                                help='max kMer size ')
        self.parser.add_argument('-p', '--pseudocount', nargs='?', type=float, default=1, action='store',
                                help='Zscore cutoff')


        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)

In [11]:
from random import randint
from math import prod, log2

"""
Author: Cade Mirchandani
Group: Gabe P., Jodie J., Hailey L.
Class: BME 205 Fall 2022
"""

class RandomMotifSearch:
    """
    Finds motif of a user-specified size in input sequence using a randomized motif search approach.
    """
    
    def __init__(self, seq, iterations, motifSize, pseudocounts) -> None:
        """
        Args:
            seq (_type_): Generator returned from FastAreader
            iterations (_type_): Number of iterations of search to complete
            motifSize (_type_): Size (bp) of motif to search for
            pseudocounts (_type_): Number of pseudocounts to use
        """
        
        self.seq = list(seq)
        self.iterations = iterations
        self.motifSize = motifSize
        self.pseudocounts = pseudocounts
        self.Q = self.qDict()
    
    def qDict(self) -> dict[str, float]:
        """
        Creates null model count dictionary

        Returns:
            dict[str, float]: Dict of counts of bases.
        """
        d = {"A": self.pseudocounts, "C": self.pseudocounts, "G": self.pseudocounts, "T": self.pseudocounts}
        tot = self.pseudocounts * 4
        for header, seq in self.seq:
            for char in seq:
                d[char] += 1
                tot += 1
        return {k: v/tot for k,v in d.items()}
    
    def generateConsensus(self, profile):
        """
        Generates consesus sequence from profile

        Args:
            profile (dict): Motif profile
        """
        consensus = ""
        score = self.scoreProfile(profile)
        for i in range(self.motifSize):
            probs = [(k ,profile[k][i]) for k in profile.keys()]
            consensus += max(probs, key=lambda x: x[1])[0]
        print(consensus, score, sep="\t")
                
    def iterateSearch(self):
        """
        Performs iterative random motif search.

        Returns:
            dict: The best profile found after the search
        """
        bestProfile = self.randomizedMotifSearch()
        for _ in range(0, self.iterations):
            newProfile  = self.randomizedMotifSearch()
            if self.scoreProfile(bestProfile) < self.scoreProfile(newProfile):
                bestProfile = newProfile
        return bestProfile
            
    def randomizedMotifSearch(self) -> dict[str, list[float]]:
        """
        Performs randomized motif search

        Returns:
            dict[str, list[float]]: The best profile found
        """
        randomMotifs = self.generateRandomMotifs()
        bestProfile = self.generateMotifProfile(randomMotifs)
        while True:
            newMotifs = self.motifsFromProfile(bestProfile)
            newProfile = self.generateMotifProfile(newMotifs)
            if self.scoreProfile(bestProfile) < self.scoreProfile(newProfile):
                bestProfile = newProfile
            else:
                return bestProfile
    
    def generateRandomMotifs(self) -> list[str]:
        """
        Generates random motifs from input sequence.

        Returns:
            list[str]: List of motifs from input seq.
        """
        motifs = []
        for header, seq in self.seq:
            randStart = randint(0, len(seq)-self.motifSize)
            motifs.append(seq[randStart : (randStart + self.motifSize)])
        return motifs
    
    def generateMotifProfile(self, motifs: list[str]) -> dict[str, list[float]]:
        """
        Generates motif profile from list of motifs

        Args:
            motifs (list[str]): List of motifs

        Returns:
            dict[str, list[float]]: Profile of input motifs
        """
        profile = {k : [self.pseudocounts] * self.motifSize for k in "ACGT"}
        for motif in motifs:
            for i, char in enumerate(motif):
                profile[char][i] += 1
        for k in profile.keys():
            profile[k] = [x / (len(self.seq) + (self.pseudocounts * 4)) for x in profile[k]]
        return profile #type: ignore
        
    def motifsFromProfile(self, profile) -> list[str]:
        """
        Generates motifs given a motif profile.

        Args:
            profile (dict): A motif profile

        Returns:
            list[str]: List of motifs
        """
        bestMotifs = []
        for header, seq in self.seq:
            bestProb = 0
            bestKmer = None
            for k in range(0, len(seq)-self.motifSize):
                kmer = seq[k : k + self.motifSize]
                kmerProb = prod([profile[char][i] for i, char in enumerate(kmer)])
                if kmerProb > bestProb:
                    bestKmer = kmer
                    bestProb = kmerProb
            bestMotifs.append(bestKmer)
        return bestMotifs
    
    def scoreProfile(self, profile) -> float:
        """
        Scores profile using relative entropy

        Args:
            profile (dict): A profile to score

        Returns:
            float: The score of the profile
        """
        entropy = 0
        for j in range(self.motifSize):
            for k in profile.keys():
                pi = profile[k][j]
                qi = self.Q[k]
                entropy += (pi * log2(pi/qi))
        return entropy
    


In [12]:
def main(inFile=None, options=None):
    ''' Setup necessary objects, read data and print the final report.'''
    cli = CommandLine(options)
    f = FastAreader(inFile)
    seq = f.readFasta()
    m = RandomMotifSearch(seq, cli.args.iterations, cli.args.motifSize, cli.args.pseudocount)
    m.generateConsensus(m.iterateSearch())
if __name__ == "__main__":   
    main("./test_data/all.fa", options = [ "-i 1000", "-k 13", "-p 1"])

GAAAAACTTAAAA	11.831405571748094
